In [1]:
import os

In [2]:
print(os.getcwd())

e:\STUDY\TENSORFLOW\Projects\1_CNN_Project\research


In [3]:
os.chdir("../")

In [4]:
os.getcwd()

'e:\\STUDY\\TENSORFLOW\\Projects\\1_CNN_Project'

In [ ]:
# Update src/cnn_classifier/entity/config_entity.py.

from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareBaseModelConfig:

    root_dir: Path
    base_model_path : Path
    custom_base_model_path: Path
    params_include_top : bool
    params_weights : str
    params_image_size : list
    params_learning_rate : float
    params_classes : int
    

In [ ]:
# Update the src/cnn_classifier/config/configuration.py file.

from cnn_classifier.constants import *
from cnn_classifier.utils.common import read_yaml, create_directories

In [ ]:
class ConfigurationManager:

    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):

        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)

        # Creating directory
        create_directories([self.config.artifacts_root])

    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:

        config=self.config.prepare_base_model

        create_directories([config.root_dir])

        prepare_base_model_config= PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            custom_base_model_path=Path(config.custom_base_model_path),
            params_include_top=self.params.INCLUDE_TOP,
            params_weights=self.params.WEIGHTS,
            params_image_size=self.params.INPUT_SHAPE,
            params_learning_rate=self.params.LEARNING_RATE,
            params_classes=self.params.CLASSES
                )
        
        return prepare_base_model_config

In [ ]:
# Update the src/cnn_classifier/components/prepare_base_model.py file

import os
from urllib import request
from zipfile import ZipFile
import tensorflow as tf
from typing import Optional


In [ ]:
class PrepareBaseModel:

    def __init__(self, config:PrepareBaseModelConfig):

        self.config=config

    @staticmethod
    def save_model(path:Path, model:tf.keras.Model):

        model.save(path)

    
    def get_base_model(self):

        self.model=tf.keras.applications.vgg16.VGG16(
            include_top=self.config.params_include_top,
            weights=self.config.params_weights,
            input_shape=self.config.params_image_size
        )

        # calling static method(save_model) to save the base model
        self.save_model(path=self.config.base_model_path,
                        model=self.model)
        
    
    @staticmethod
    def _prepare_full_model(model,
                            classes,           
                            freeze_all:bool,
                            freeze_till:Optional[int],
                            learning_rate,     # option is given but not used
                            input_shape):

        """"
        Optional[int] means the parameter can be either int or None.
        """
        
        # Freeze all layers if freeze_all is True
        if freeze_all:
            model.trainable = False  # Correctly set layer.trainable
        
        # Freeze layers up to a specific number if freeze_till is provided
        elif (freeze_till is not None) and (freeze_till > 0):
            for layer in model.layers[:-freeze_till]:
                layer.trainable = False  # Correctly set layer.trainable

        # Creating new model on top
        inputs=tf.keras.Input(shape=input_shape, name="input_layer")

        # Apply Rescaling
        rescale_layer=tf.keras.layers.Rescaling(scale=1/255., name="rescaling_layer")
        x = rescale_layer(inputs)

        # VGG16 model
        x = model(x, training=False)

        # Flatten layer
        flatten_layer = tf.keras.layers.Flatten(name="flatten_layer")(x)

        # Hidden layers
        # hidden_layer1 = tf.keras.layers.Dense(
        #                                     units=10,
        #                                     activation="sigmoid",
        #                                     name="hidden_layer1"
        #                                     )(flatten_layer)

        
        # hidden_layer2=tf.keras.layers.Dense(units=5, activation="relu", name="hidden_layer2")(hidden_layer1)

        #output_layer=tf.keras.layers.Dense(units=classes-1,
        #                                   activation="sigmoid",
        #                                   name="output_layer")(hidden_layer2)

        # If we want to use only single hidden layer with output layer (uncomment below code)
        #hidden_layer1=tf.keras.layers.Dense(units=5, activation="relu", name="hidden_layer1")(flatten_layer)

        output_layer=tf.keras.layers.Dense(units=classes,
                                           activation="softmax",
                                           name="output_layer")(flatten_layer)

        full_model = tf.keras.models.Model(
                                  inputs=inputs,
                                  outputs=output_layer
                                  )
        
        full_model.summary()
        
        return full_model

        
    def custom_base_model(self):

        self.full_model=self._prepare_full_model(
            model=self.model,
            classes=self.config.params_classes,
            freeze_all=True,
            freeze_till=None,
            learning_rate=self.config.params_learning_rate,
            input_shape=self.config.params_image_size

            )
        
        self.save_model(path=self.config.custom_base_model_path,
                        model=self.full_model)


In [ ]:
# Pipeline : update the src/cnn_classifier/pipeline/Stage02_prepare_base_model.py file

try:
    config=ConfigurationManager()
    prepare_base_model_config=config.get_prepare_base_model_config()
    prepare_base_model=PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.custom_base_model()

except Exception as e:
    raise e

[2025-04-25 11:09:12,756 : INFO : common : yaml file : config\config.yaml loaded successfully]
[2025-04-25 11:09:12,760 : INFO : common : yaml file : params.yaml loaded successfully]
[2025-04-25 11:09:12,761 : INFO : common : Created directory at : artifacts]
[2025-04-25 11:09:12,761 : INFO : common : Created directory at : artifacts/prepare_base_model]


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ rescaling_layer (Rescaling)     │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ vgg16 (Functional)              │ (None, 7, 7, 512)      │    14,714,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_layer (Flatten)         │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output_layer (Dense)            │ (None, 1)              │        25,089 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,739,777 (56.23 MB)

 Trainable params: 25,089 (98.00 KB)

 Non-trainable params: 14,714,688 (56.13 MB)